# Review Instructions

Please review the MSv4 processing_set class https://github.com/casangi/xradio/blob/main/src/xradio/vis/_processing_set.py

The processing set is a loose collection of MSv4 which might come from multiple MSv2 (or ASDMS). Consequently, arbitrary ids are avoided in favor of descriptive strings.

Run the notebook using:
- ```partition_scheme=['FIELD_ID']``` 
and 
- ```partition_scheme=[]``` 

## Key Questions to Answer
1) Is there additional information to display in the summary table?
2) Are the docstrings sufficient?
3) Are there missing data selection use cases?
4) ...

# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 213-fix-ps-selection
cd xradio
pip install -e .
```

On macOS it is required to pre-install python-casacore using ```bash conda install -c conda-forge python-casacore```.

# Download Data

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

[2024-08-15 10:48:36,976]  WARNING  graphviper:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-15 10:48:36,976]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List                                
 ───────────────────────────────────────────── 
  ALMA_uid___A002_X1003af4_X75a3.split.avg.ms

[2024-08-15 10:48:38,578]     INFO  graphviper:  File exists: ALMA_uid___A002_X1003af4_X75a3.split.avg.ms 


# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-08-15 10:48:38,673]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-08-15 10:48:38,673]     INFO  graphviper:  Module path: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11//site-packages/ 
[2024-08-15 10:48:38,676]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 


/Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54357 instead
  warnings.warn(


[2024-08-15 10:48:39,428]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:54358' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:54357/status,
Dashboard: http://127.0.0.1:54357/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54358,Workers: 4
Dashboard: http://127.0.0.1:54357/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:54369,Total threads: 1
Dashboard: http://127.0.0.1:54371/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:54361,


# Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
import os

in_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.ms"
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"
os.system("rm -rf "+out_file)

partition_scheme=['FIELD_ID'] #can be ephemeris_interpolate=True/False, Default
#partition_scheme=[] #must be ephemeris_interpolate=True, Rapid OTF mode

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=True,
    overwrite=True,
    ephemeris_interpolate=True,
    partition_scheme=partition_scheme
)

[2024-08-15 10:48:41,518]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID'] 
[2024-08-15 10:48:41,578]     INFO      client:  Number of partitions: 18 
[2024-08-15 10:48:41,579]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [7] 
[2024-08-15 10:48:41,583]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [16], FIELD [0], SCAN [7] 
[2024-08-15 10:48:41,585]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [17], FIELD [0], SCAN [7] 
[2024-08-15 10:48:41,585]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [19], FIELD [2], SCAN [6] 
[2024-08-15 10:48:41,586]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [20 99], FIELD [0], SCAN [8] 
[2024-08-15 10:48:41,587]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44
 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68
 69 7

# Inspect Processing Set

In [4]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_00,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_01,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
11,ALMA_uid___A002_X1003af4_X75a3.split.avg_02,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
12,ALMA_uid___A002_X1003af4_X75a3.split.avg_03,"[CALIBRATE_PHASE#ON_SOURCE, CALIBRATE_WVR#ON_S...","(31, 51, 1, 2)","[XX, YY]",[6],X767114449#ALMA_RB_06#BB_4#SQLD_0,[J1408-0752_2],[J1408-0752_2],[],"[icrs, 14h08m56.48s, -7d52m26.67s]",2.480000e+11,2.480000e+11
13,ALMA_uid___A002_X1003af4_X75a3.split.avg_04,[OBSERVE_TARGET#OFF_SOURCE],"(14, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
10,ALMA_uid___A002_X1003af4_X75a3.split.avg_05,[OBSERVE_TARGET#ON_SOURCE],"(312, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_06,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_07,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_08,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_09,"[CALIBRATE_PHASE#ON_SOURCE, CALIBRATE_WVR#ON_S...","(26, 51, 4, 1)",[XX],[6],WVR#NOMINAL_1,[J1408-0752_2],[J1408-0752_2],[],"[icrs, 14h08m56.48s, -7d52m26.67s]",1.845500e+11,1.905500e+11


# Using ps.sel() with summary table column names

In [5]:
#Note that no selection is applied on the MS data so even if field_name=['Sun_10_10','Sun_10_11'] all the fields are kept.
ps.sel(field_coords='Ephemeris',field_name=['Sun_10_10','Sun_10_11']).summary() #Select all Ephemeris data and where any of the fields are 'Sun_10_10' or 'Sun_10_11'.



,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_05,[OBSERVE_TARGET#ON_SOURCE],"(312, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_11,[OBSERVE_TARGET#ON_SOURCE],"(234, 51, 4, 1)",[XX],[8],WVR#NOMINAL_1,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_17,[OBSERVE_TARGET#ON_SOURCE],"(234, 1326, 7, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_1#SW-01#FULL_RES_2,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[Single_Continuum(ID=0)],Ephemeris,2.299609e+11,2.300547e+11


# Using ps.sel() with query and summary table column names

In [6]:
ps.sel(query="start_frequency > 2.46e11",field_coords='Ephemeris',field_name=['Sun_10_10','Sun_10_11']).summary() #Select all Ephemeris data and where any of the fields are 'Sun_10_10' or 'Sun_10_11'.

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_05,[OBSERVE_TARGET#ON_SOURCE],"(312, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11


# Ordering MSv4 Selection

In [7]:
summary_df = ps.sel(obs_mode='OBSERVE_TARGET#ON_SOURCE').summary()
summary_df=summary_df.sort_values(by=['start_frequency'],ascending=True)

summary_df

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_11,[OBSERVE_TARGET#ON_SOURCE],"(234, 51, 4, 1)",[XX],[8],WVR#NOMINAL_1,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_17,[OBSERVE_TARGET#ON_SOURCE],"(234, 1326, 7, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_1#SW-01#FULL_RES_2,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[Single_Continuum(ID=0)],Ephemeris,2.299609e+11,2.300547e+11
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_05,[OBSERVE_TARGET#ON_SOURCE],"(312, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11


In [8]:
#First MS is then give by:
first_ms_name = summary_df['name'][0]
ps[first_ms_name]

<xarray.Dataset> Size: 1MB
Dimensions:                     (time: 312, baseline_id: 51, frequency: 1,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    antenna_id                  (baseline_id) int32 204B dask.array<chunksize=(51,), meta=np.ndarray>
    baseline_antenna1_name      (baseline_id) <U9 2kB dask.array<chunksize=(51,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U9 2kB dask.array<chunksize=(51,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 408B 0 1 2 3 ... 47 48 49 50
  * frequency                   (frequency) float64 8B 2.48e+11
    mount                       (baseline_id) <U6 1kB dask.array<chunksize=(51,), meta=np.ndarray>
    name                        (baseline_id) <U9 2kB dask.array<chunksize=(51,), meta=np.ndarray>
  * polarization                (polarization) <U2 16B 'XX' 'YY'
    scan_number                 (time) int64 2kB dask.array<chunksize=(312,), meta=np.ndarray>
    station                     (baseline_id) <U4 816B dask.array<chunksize=(51,), meta=np.ndarray>
  * time                        (time) float64 2kB 1.667e+09 ... 1.667e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 127kB dask.array<chunksize=(312, 51), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 32kB dask.array<chunksize=(312, 51, 1, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 127kB dask.array<chunksize=(312, 51), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 382kB dask.array<chunksize=(312, 51, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 255kB dask.array<chunksize=(312, 51, 1, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 127kB dask.array<chunksize=(312, 51, 1, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_name': ['Sun_10_10', 'Sun_10_11', 'Sun_10_12', '...
    pointing_xds:    <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variabl...
    weather_xds:     <xarray.Dataset> Size: 138kB\nDimensions:         (stati...
    antenna_xds:     <xarray.Dataset> Size: 10kB\nDimensions:                ...

# Sel by number

In [9]:

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

min_freq = min(ps.summary()['start_frequency'])
ps.sel(start_frequency=min_freq).summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_06,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_07,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_08,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_09,"[CALIBRATE_PHASE#ON_SOURCE, CALIBRATE_WVR#ON_S...","(26, 51, 4, 1)",[XX],[6],WVR#NOMINAL_1,[J1408-0752_2],[J1408-0752_2],[],"[icrs, 14h08m56.48s, -7d52m26.67s]",1.845500e+11,1.905500e+11
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_10,[OBSERVE_TARGET#OFF_SOURCE],"(11, 51, 4, 1)",[XX],[8],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_11,[OBSERVE_TARGET#ON_SOURCE],"(234, 51, 4, 1)",[XX],[8],WVR#NOMINAL_1,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11


# Require exact match in selection criteria

In [10]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.sel(name='ALMA_uid___A002_X1003af4_X75a3.split.avg_01',string_exact_match=True).summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_01,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11


# Allow for partial match in string. (select all MSv4 with field_names with "Sun_10" in the name).

In [11]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.sel(field_name='Sun_10',string_exact_match=False).summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_00,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_01,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
9,ALMA_uid___A002_X1003af4_X75a3.split.avg_02,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
10,ALMA_uid___A002_X1003af4_X75a3.split.avg_04,[OBSERVE_TARGET#OFF_SOURCE],"(14, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
8,ALMA_uid___A002_X1003af4_X75a3.split.avg_05,[OBSERVE_TARGET#ON_SOURCE],"(312, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_06,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_07,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_08,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
12,ALMA_uid___A002_X1003af4_X75a3.split.avg_10,[OBSERVE_TARGET#OFF_SOURCE],"(11, 51, 4, 1)",[XX],[8],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
13,ALMA_uid___A002_X1003af4_X75a3.split.avg_11,[OBSERVE_TARGET#ON_SOURCE],"(234, 51, 4, 1)",[XX],[8],WVR#NOMINAL_1,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11


# Allow for partial match in string. (select all MSv4 with ALMA_uid___A002_X1003af4_X75a3.split.avg in the name).

This example does nothing but would be useful for a PS created from MSv4s that are created from different MSv2s.

In [12]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.sel(name='ALMA_uid___A002_X1003af4_X75a3.split.avg',string_exact_match=False).summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_00,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_01,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
11,ALMA_uid___A002_X1003af4_X75a3.split.avg_02,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(3, 51, 1, 2)","[XX, YY]",[7],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
12,ALMA_uid___A002_X1003af4_X75a3.split.avg_03,"[CALIBRATE_PHASE#ON_SOURCE, CALIBRATE_WVR#ON_S...","(31, 51, 1, 2)","[XX, YY]",[6],X767114449#ALMA_RB_06#BB_4#SQLD_0,[J1408-0752_2],[J1408-0752_2],[],"[icrs, 14h08m56.48s, -7d52m26.67s]",2.480000e+11,2.480000e+11
13,ALMA_uid___A002_X1003af4_X75a3.split.avg_04,[OBSERVE_TARGET#OFF_SOURCE],"(14, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,[Sun_10_0],[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
10,ALMA_uid___A002_X1003af4_X75a3.split.avg_05,[OBSERVE_TARGET#ON_SOURCE],"(312, 51, 1, 2)","[XX, YY]",[8],X767114449#ALMA_RB_06#BB_4#SQLD_0,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,2.480000e+11,2.480000e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_06,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_07,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_08,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(2, 51, 4, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.905500e+11
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_09,"[CALIBRATE_PHASE#ON_SOURCE, CALIBRATE_WVR#ON_S...","(26, 51, 4, 1)",[XX],[6],WVR#NOMINAL_1,[J1408-0752_2],[J1408-0752_2],[],"[icrs, 14h08m56.48s, -7d52m26.67s]",1.845500e+11,1.905500e+11


In [13]:
ps.sel(spw_name=ps.summary().spw_name[0]).ms_isel(frequency=0).summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_06,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(2, 51, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.845500e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_07,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(2, 51, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.845500e+11
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_08,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(2, 51, 1)",[XX],[7],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.845500e+11
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_09,"[CALIBRATE_PHASE#ON_SOURCE, CALIBRATE_WVR#ON_S...","(26, 51, 1)",[XX],[6],WVR#NOMINAL_1,[J1408-0752_2],[J1408-0752_2],[],"[icrs, 14h08m56.48s, -7d52m26.67s]",1.845500e+11,1.845500e+11
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_10,[OBSERVE_TARGET#OFF_SOURCE],"(11, 51, 1)",[XX],[8],WVR#NOMINAL_1,[Sun_10_0],[Sun_10_0],[],Ephemeris,1.845500e+11,1.845500e+11
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_11,[OBSERVE_TARGET#ON_SOURCE],"(234, 51, 1)",[XX],[8],WVR#NOMINAL_1,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,1.845500e+11,1.845500e+11


In [14]:
ps.sel(spw_name=ps.summary().spw_name[0]).ms_isel(frequency=0).sel(field_name='Sun_10_29').summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_11,[OBSERVE_TARGET#ON_SOURCE],"(234, 51, 1)",[XX],[8],WVR#NOMINAL_1,"[Sun_10_10, Sun_10_11, Sun_10_12, Sun_10_13, S...",[Sun_10_0],[],Ephemeris,1.845500e+11,1.845500e+11
